In [2]:
import random
from Crypto.Hash import SHA
#Note:用于生成安全哈希和消息摘要。
import math
class DSA():
    """
    签名和验证
    """
    def __init__(self):
        pass
    #Note:由于当前类的初始化逻辑尚未明确，因此初始化函数暂时留空.
    #Note:生成全局公钥和私钥.
    def Globalpublickey(self):
        """
            p:满足2^(L-1)<p<2^L的大素数 其中512<=L<=1024 且L为64的倍数
            q:p-1的素因子,满足2^159<2^160,即q长160比特
            g:g=h^((p-1)/q)mod p,h是满足1<h<p-1且h^((p-1)/q)mod p>1的任意整数
        """
        n=160   #Note:q的长度。
        while True:
            Q=random.randrange(2**(n-1)+1,2**n-1,2)
            #Note:判断是否为素数。
            if self.MillerRabin(Q):
                self.q=Q
                break 
        while True:
            i=random.randint(0,8)   #Note:满足L是64的倍数
            L=512+i*64  #Note:p的长度（满足 512 <= L <= 1024）。
            k=random.randint((2**(L-160)-2**(L-163)),(2**(L-160)+2**(L-163)))
            p=k*self.q+1    #Note:生成一个随机数k满足该条件。
            #Note:判断是否为素数。
            if self.MillerRabin(p):
                self.p=p
                break
        while True:
            h=random.randint(2,self.p-2)    #Note:生成一个随机数h满足1<h<p-2 。
            #Note:print("shdjsa",(self.p-1)//self.q,k)。
            g=self.modExp(h,(self.p-1)//self.q,self.p)  #Note:生成一个随机数g满足g=h^((p-1)/q)mod p。
            if g>1:
                self.g=g
                break
    
    def UserKey(self):
        #Note:生成私钥和公钥.
        x=random.randint(1,self.q-1)
        #Note:生成一个随机整数 x，作为用户的私钥。
        self.x = x #Usersecret
        self.y=self.modExp(self.g,self.x,self.p)
        #Note:使用 modExp 方法计算公钥 y，公式为 y = g^x mod p  。
        #Note:Publickey。
    
    def MillerRabin(self,n):#素数检测算法
        """
        判断一个数是否是素数，使用Miller-Rabin概率检测法
        n:待检验的数
        a:小于n的数
        如果n暂时测试为素数，则返回true
        """
        if n<2:
            return False
        if n<=3:
            return True
        binstr = bin(n-1)[2:]   #Note:将 n-1 转换为二进制字符串 binstr。
        for i in range(8):
            a = random.randint(2, n)
            d = 1
            #Note:对 a 进行模幂运算，检查中间结果 d 是否满足特定条件。如果条件不满足，则返回 False。
            for i in binstr:
                x = d
                d = d*d%n
                if d==1 and x!=1 and x!=n-1:
                    return False
                if i=='1':
                    d = d*a%n
            if d!=1:
                return False
        return True

    def modExp(self,a,b,n):
        """
        快速幂取模,a^b%n
        args{
            a:底数
            b:指数
            n:取模数
        }
        """
        ans=1
        #Note:使用位运算进行快速幂运算。
        while b:
            if b&1:
                ans=self.modMul(ans,a,n)
            a=self.modMul(a,a,n)
            b>>=1
        #Note:如果 b 的最低位为 1（即 b & 1 为真），则将当前的 ans 与 a 相乘后取模 n，结果存回 ans。
        #Note:将 a 乘以 2，取模 n，结果存回 ans。
        #Note:将 b 右移一位（相当于除以 2）。
        return ans
        
    def modMul(self,a,b,n):
        """
        快速积取模,a*b%n
        args{
            a:乘数
            b:乘数
            n:取模数
        }
        """
        ans=0
        while b:    #Note:遍历b的每一位。
            if b&1: #Note:如果 b 的当前位为 1，则将 a 加到 ans 上，并对 n 取模。
                ans=(ans+a)%n
                #Note:将 a 左移一位（相当于 a 乘以 2），并对 n 取模。
            a=(a+a)%n
            #Note:将 b 右移一位（相当于 b 除以 2）。
            b>>=1
        return ans

    def Inverse(self, a,mod):
        #Note:求秘钥a的逆元——欧几里得算法。   
        x1,x2,x3 = 1,0,mod
        y1,y2,y3 = 0,1,a
        while(1):
            #Note:使用扩展欧几里得算法进行迭代，直到 y3 为 0 或 1。
            if(y3==0):
                g=x3
                break
            if(y3==1):
                g=y3
                break
            q=math.floor(x3/y3)#向下取整
            t1,t2,t3=x1-q*y1,x2-q*y2,x3-q*y3
            x1,x2,x3=y1,y2,y3
            y1,y2,y3=t1,t2,t3
        #Note:如果 y2 小于 0，则将其加上 mod。
        if y2<0:
            y2+=mod
        return y2   #逆元求得为y2,y3为gcd(a,26),最大公因数 

    def signature(self):
        self.k=random.randint(1,self.q-1)
        self.k_Reverse=self.Inverse(self.k,self.q)
        #Note:生成随机数k，范围在 [1, q-1] 之间，并求出k的逆元。
        print("用户为待签消息选取的秘密数k：",self.k)
        print("秘钥数k的逆元：",self.k_Reverse)
        choice=int(input("请选择待签数据类型-对字符串签名[0],对文件签名[1]:"))
        #Note:选择待签数据类型（字符串或文件）。
        if choice==0:
            message=input("请输入待签消息：")
            self.data=message
            Hash=SHA.new(message.encode("utf8")).hexdigest()
            self.Hash=int(Hash,16)
            self.r=self.modExp(self.g,self.k,self.p)%self.q
            self.s=self.modMul(self.k_Reverse,self.Hash+self.x*self.r,self.q)
            print("用户对消息的签名为:(r,s)\nr:{}\ns:{}".format(self.r,self.s))
        #Note:对字符串进行签名：用户输入待签消息——计算消息的哈希值——使用 DSA 算法生成签名 (r, s)。    
        elif choice==1:
            filename=input("请输入文件：")
            with open(filename,'rb') as fp:
                if not fp:
                    print("请输入正确文件名！")
                    exit(0)
                data=fp.read()
            self.data=data
            Hash=SHA.new(data).hexdigest()
            self.Hash=int(Hash,16)
            self.r=self.modExp(self.g,self.k,self.p)%self.q
            self.s=self.modMul(self.k_Reverse,self.Hash+self.x*self.r,self.q)
            print("用户对消息的签名为:(r,s)\nr:{}\ns:{}".format(self.r,self.s))
        #Note:对文件进行签名：打开文件——计算文件的哈希值——使用 DSA 算法生成签名 (r, s)。
        return choice
    
    def verification(self,message,flag=0):
        #Note:根据 flag 的值选择不同的方式计算消息的哈希值。
        #Note:如果 flag 为 0，则将消息编码为 UTF-8 后计算哈希；如果 flag 为 1，则直接计算消息的哈希。
        if flag==0:
            Hash=SHA.new(message.encode("utf8")).hexdigest()
        elif flag==1:
            Hash=SHA.new(message).hexdigest()
        #print("验证",message,Hash)
        #print("签名",self.data,hex(self.Hash).replace("0x",""))
        #Note:将计算得到的哈希值从十六进制字符串转换为整数。
        Hash=int(Hash,16)
        print("接受到的消息的哈希值：",Hash)
        #Note:计算 w、u1 和 u2，这些变量用于后续的验证计算。
        w=self.Inverse(self.s,self.q)
        u1=self.modMul(Hash,w,self.q)
        u2=self.modMul(self.r,w,self.q)
        #Note:使用 modExp 和 modMul 方法计算 v，这是验证签名的关键步骤。
        v=self.modMul(self.modExp(self.g,u1,self.p),self.modExp(self.y,u2,self.p),self.p)%self.q
        #v1=hex(v).replace("0x","")
        #print("v:",v1,len(v1))
        #print("r:",hex(self.r).replace("0x",""))
        #print(len(str(bin(self.q).replace("0b",""))),len(str(bin(self.p).replace("0b",""))))
        #print(v,self.r)
        #Note:比较 v 和 self.r，如果相等则签名有效，否则签名无效。
        if v==self.r:
            print("签名有效！")
        else:
            print("签名无效！")
              
if __name__ == "__main__":
    test=DSA()  #Note:创建 DSA 类的实例 test。
    #Note:生成全局公钥（Globalpublickey 方法）和用户密钥（UserKey 方法）。
    test.Globalpublickey()
    test.UserKey()
    print("全局公开钥\nP:{}\nQ:{}\nG:{}".format(test.p,test.q,test.g))
    print("用户密钥\n秘密钥x:{}\n公开钥y:{}".format(test.x,test.y))
    #Note:对消息进行签名（signature 方法），并打印消息的哈希值。
    opntion=test.signature()
    print("消息哈希值为",test.Hash)
    #Note验证签名（verification 方法）。
    test.verification(test.data,opntion)
    while True:
        flag=input("是否继续验证？[y][n]:")
        if flag.lower()=='y':
            #opntion=int(input("验证签名的消息类型-[0]字符串[1]文件："))
            #test.signature()
            if opntion==0:
                message=input("验证消息：")
                test.verification(message)
            elif opntion==1:
                filename=input("请输入文件：")
                with open(filename,'rb') as fp:
                    if not fp:
                        print("请输入正确文件名！")
                        continue
                    data=fp.read()
                test.verification(data,opntion)
        elif flag.lower()=='n':
            break


全局公开钥
P:178155907048764811980002912598284616359508741188328738695015065296246618674965766244485172997155614740861632188198521970360830408302603825318445881147738270263423797428541151
Q:1156033909042075984553761754065236228068560505951
G:151304001820705847019671537259897036879288887863667274322488337808151526528071719044745227862192200822032669524466747280296743125980540637242658364897108463554666807869511737
用户密钥
秘密钥x:973554643749008817426529911192471448916166712315
公开钥y:54979093651144557641206521379975517176526479496806224516288862610173533231587077907332900112482846742925225951792588384021764428963723698257106681504986020468481864346867493
用户为待签消息选取的秘密数k： 72168421774518680713382858591554046565582625797
秘钥数k的逆元： 439929045529261020432356177903174417800708812876
请选择待签数据类型-对字符串签名[0],对文件签名[1]:0
请输入待签消息：132456798
用户对消息的签名为:(r,s)
r:520361509224269803163378694917636856567133965642
s:663156638299541235002637063022660386641710810899
消息哈希值为 679058421204877065672834466204875035054000595496
接受到的消

In [3]:
import binascii

from gmssl import sm2 as SM2
from random import SystemRandom
from base64 import b64encode, b64decode

from gmssl.func import random_hex


class CurveFp:
    def __init__(self, A, B, P, N, Gx, Gy, name):#参数初始化
        self.A = A
        self.B = B
        self.P = P
        self.N = N
        self.Gx = Gx
        self.Gy = Gy
        self.name = name


class SM2Key:
    #给出参数对象
    sm2p256v1 = CurveFp(
        name="sm2p256v1",
        A=0xFFFFFFFEFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF00000000FFFFFFFFFFFFFFFC,
        B=0x28E9FA9E9D9F5E344D5A9E4BCF6509A7F39789F515AB8F92DDBCBD414D940E93,
        P=0xFFFFFFFEFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF00000000FFFFFFFFFFFFFFFF,
        N=0xFFFFFFFEFFFFFFFFFFFFFFFFFFFFFFFF7203DF6B21C6052B53BBF40939D54123,
        Gx=0x32C4AE2C1F1981195F9904466A39C9948FE30BBFF2660BE1715A4589334C74C7,
        Gy=0xBC3736A2F4F6779C59BDCEE36B692153D0A9877CC62A474002DF32E52139F0A0
    )

    @staticmethod
    def multiply(a, n, N, A, P):
        #椭圆曲线的乘法运算
        return SM2Key.fromJacobian(SM2Key.jacobianMultiply(SM2Key.toJacobian(a), n, N, A, P), P)

    @staticmethod
    def add(a, b, A, P):
        #椭圆曲线的加法运算
        return SM2Key.fromJacobian(SM2Key.jacobianAdd(SM2Key.toJacobian(a), SM2Key.toJacobian(b), A, P), P)

    @staticmethod
    def inv(a, n):
        #计算模逆元
        if a == 0:
            return 0
        lm, hm = 1, 0
        low, high = a % n, n
        while low > 1:
            r = high // low
            nm, new = hm - lm * r, high - low * r
            lm, low, hm, high = nm, new, lm, low
        return lm % n

    @staticmethod
    def toJacobian(Xp_Yp):
        #坐标转换
        Xp, Yp = Xp_Yp
        return Xp, Yp, 1

    @staticmethod
    def fromJacobian(Xp_Yp_Zp, P):
        #转变为仿射坐标系
        Xp, Yp, Zp = Xp_Yp_Zp
        z = SM2Key.inv(Zp, P)
        return (Xp * z ** 2) % P, (Yp * z ** 3) % P

    @staticmethod
    def jacobianDouble(Xp_Yp_Zp, A, P):
        #计算点的二倍
        Xp, Yp, Zp = Xp_Yp_Zp
        if not Yp:
            return 0, 0, 0
        ysq = (Yp ** 2) % P
        S = (4 * Xp * ysq) % P
        M = (3 * Xp ** 2 + A * Zp ** 4) % P
        nx = (M ** 2 - 2 * S) % P
        ny = (M * (S - nx) - 8 * ysq ** 2) % P
        nz = (2 * Yp * Zp) % P
        return nx, ny, nz

    @staticmethod
    def jacobianAdd(Xp_Yp_Zp, Xq_Yq_Zq, A, P):
        #椭圆曲线点加法
        Xp, Yp, Zp = Xp_Yp_Zp
        Xq, Yq, Zq = Xq_Yq_Zq
        if not Yp:
            return Xq, Yq, Zq
        if not Yq:
            return Xp, Yp, Zp
        U1 = (Xp * Zq ** 2) % P
        U2 = (Xq * Zp ** 2) % P
        S1 = (Yp * Zq ** 3) % P
        S2 = (Yq * Zp ** 3) % P
        if U1 == U2:
            if S1 != S2:
                return 0, 0, 1
            return SM2Key.jacobianDouble((Xp, Yp, Zp), A, P)
        H = U2 - U1
        R = S2 - S1
        H2 = (H * H) % P
        H3 = (H * H2) % P
        U1H2 = (U1 * H2) % P
        nx = (R ** 2 - H3 - 2 * U1H2) % P
        ny = (R * (U1H2 - nx) - S1 * H3) % P
        nz = (H * Zp * Zq) % P
        return nx, ny, nz

    @staticmethod
    def jacobianMultiply(Xp_Yp_Zp, n, N, A, P):
        #椭圆曲线点乘法
        Xp, Yp, Zp = Xp_Yp_Zp
        if Yp == 0 or n == 0:
            return (0, 0, 1)
        if n == 1:
            return (Xp, Yp, Zp)
        if n < 0 or n >= N:
            return SM2Key.jacobianMultiply((Xp, Yp, Zp), n % N, N, A, P)
        if (n % 2) == 0:
            return SM2Key.jacobianDouble(SM2Key.jacobianMultiply((Xp, Yp, Zp), n // 2, N, A, P), A, P)
        if (n % 2) == 1:
            mv = SM2Key.jacobianMultiply((Xp, Yp, Zp), n // 2, N, A, P)
            return SM2Key.jacobianAdd(SM2Key.jacobianDouble(mv, A, P), (Xp, Yp, Zp), A, P)


class PrivateKey:
    def __init__(self, curve=SM2Key.sm2p256v1, secret=None):
        self.curve = curve
        self.secret = secret or SystemRandom().randrange(1, curve.N)
        print(self.secret)

    def PublicKey(self):
        curve = self.curve
        xPublicKey, yPublicKey = SM2Key.multiply((curve.Gx, curve.Gy), self.secret, A=curve.A, P=curve.P, N=curve.N)
        return PublicKey(xPublicKey, yPublicKey, curve)

    def ToString(self):
        #改变secret属性，即转化为一个64位16进制字符串。
        return "{}".format(str(hex(self.secret))[2:].zfill(64))


class PublicKey:
    def __init__(self, x, y, curve):
        self.x = x
        self.y = y
        self.curve = curve

    def ToString(self, compressed=True):
        #根据是否进行过压缩操作来选择改变后的格式。
        return '04' + {
            True: str(hex(self.x))[2:],
            False: "{}{}".format(str(hex(self.x))[2:].zfill(64), str(hex(self.y))[2:].zfill(64))
        }.get(compressed)


class SM2Util:
    def __init__(self, pub_key=None, pri_key=None):
        self.pub_key = pub_key
        self.pri_key = pri_key
        self.sm2 = SM2.CryptSM2(public_key=self.pub_key, private_key=self.pri_key)

    def Encrypt(self, data):
        #加密数据
        info = self.sm2.encrypt(data.encode())
        return b64encode(info).decode()

    def Decrypt(self, data):
        #解密数据
        info = b64decode(data.encode())
        return self.sm2.decrypt(info).decode()

    def Sign(self, data):
        #签名算法
        # random_hex_str = random_hex(self.sm2.para_len)
        """有要求配置ID可配置"""
        random_hex_str = '1234567812345678'.encode('utf-8').hex()
        #根据特定需求设置一个十六进制字符串。
        self.sm2_crypt = SM2.CryptSM2(public_key=self.pub_key, private_key=self.pri_key)
        #封装了 SM2 算法相关的加密、解密、签名等操作的实现类的实例，用于后续对数据进行签名和验签操作。
        sign = self.sm2_crypt.sign_with_sm3(data.encode(), random_hex_str)
        #使用创建好的SM2.CryptSM2对象的sign_with_sm3方法对数据进行签名。
        print('sign:%s' % sign)
        # 验签
        verify = self.sm2_crypt.verify_with_sm3(sign, data.encode())
        #使用同一个SM2.CryptSM2对象的verify_with_sm3方法对刚刚生成的签名进行验证。
        print('verify:%s' % verify)

        return sign

    def Verify(self, data, sign):
        return self.sm2_crypt.verify_with_sm3(sign, data.encode())

    @staticmethod
    def GenKeyPair(secret):
        #生成密钥对分别提取pri与pub，随后转变为字符串格式并返回。
        pri = PrivateKey(secret=secret)
        pub = pri.PublicKey()
        return pri.ToString(), pub.ToString(compressed=False)


if __name__ == '__main__':
    data = 'hahahaa12323'
    print('原数据:{}'.format(data))
    secret_int = None
    # secret_int = int('54a1edf8a404fa8e52dc2c6d37d7bbe0bf915f85e85a0af350478271e5f60cd3', 16)


    e = SM2Util.GenKeyPair(secret_int)
    print('私钥:{} 公钥:{}'.format(e[0], e[1]))
    sm2 = SM2Util(pri_key=e[0], pub_key=e[1][2:])

    sign = sm2.Sign(data)
    print('签名:{} 验签:{}'.format(sign, sm2.Verify(data, sign)))

    cipher = sm2.Encrypt(data)
    print('加密:{}\n解密:{}'.format(cipher, sm2.Decrypt(cipher)))

原数据:hahahaa12323
101999704440073735906906562596004676674668839391731526213194930791446640780037
私钥:e181c96ec6325aea9ed052418e89e07363435a83d17f0e268ac03723bbe6e305 公钥:04215a683e82f6de16478da489c0907a05b6cc47348b71cf0f29d046352a7d299aa91da1ca91a3012824d53a05eb37659a7e3292676d6d147c39d5021c8fce46a7
sign:94b760209511c44f662ed8660e8b3f4343f4f20881f2da6e6cda045433172699622e334baec72e2dba0eb52f9f6bf1bc4f26d77f258611efa31397219fe6c5d0
verify:True
签名:94b760209511c44f662ed8660e8b3f4343f4f20881f2da6e6cda045433172699622e334baec72e2dba0eb52f9f6bf1bc4f26d77f258611efa31397219fe6c5d0 验签:True
加密:z03JVCIS0iMSNSo9gH6Q7F9kosIfOCuYVG/Ug7rVSWrwCK4Dc+98A4XKQb93NtQ35BSp2NmcM8++TNmMyjDTmztQHMUXx3zpAET2S6Ex4nNm7Iaf5quHIfLR9wCILwHD3OfqurUF8Tp5nwF1
解密:hahahaa12323
